In [7]:
import os
import torch
import torchaudio
import numpy as np
from scipy.spatial.distance import cosine

# 🔧 Settings
STORED_AUDIO_DIR = "/content/audiofiles"
UPLOAD_FILE = "output.wav" # Changed from "Siya.wav" to "output.wav"
FEATURE_SIZE = 64 # Define a fixed size for feature vectors

# 🎵 Load audio and get mel-spectrogram
def extract_features(filepath):
    waveform, sample_rate = torchaudio.load(filepath)
    transform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=64)
    mel_spec = transform(waveform)
    features = torch.mean(mel_spec, dim=-1).squeeze().numpy()  # shape: [n_mels]

    # Ensure the features are of a fixed size
    if features.ndim > 1:
        features = features.flatten()

    # Pad or truncate features to the fixed size
    if features.shape[0] < FEATURE_SIZE:
        features = np.pad(features, (0, FEATURE_SIZE - features.shape[0]))
    elif features.shape[0] > FEATURE_SIZE:
        features = features[:FEATURE_SIZE]

    return features

# 💾 Load stored audio features
def load_audio_database():
    database = []
    # Create the directory if it doesn't exist
    os.makedirs(STORED_AUDIO_DIR, exist_ok=True)
    for fname in os.listdir(STORED_AUDIO_DIR):
        if fname.endswith(".wav"):
            path = os.path.join(STORED_AUDIO_DIR, fname)
            features = extract_features(path)
            database.append((fname, features))
    return database

# 🔍 Compare and rank similarities
def compare_uploaded_to_database(upload_path):
    upload_features = extract_features(upload_path)
    database = load_audio_database()

    similarities = []
    for name, features in database:
        score = 1 - cosine(upload_features, features)  # cosine similarity
        similarities.append((name, score))

    # Sort by descending similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities

# 🚀 Run
if __name__ == "__main__":
    results = compare_uploaded_to_database(UPLOAD_FILE)
    print("🔍 Similar audio files:")
    for name, score in results[:5]:  # top 5
        print(f"{name}: similarity score = {score:.3f}")

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

🔍 Similar audio files:
NewSample.wav: similarity score = 0.825
Siya.wav: similarity score = 0.765


Please upload the audio file "Siya.wav".

In [6]:
from google.colab import files

uploaded = files.upload()

# Assuming the uploaded file is 'Siya.wav'
if 'Siya.wav' in uploaded:
  print("File 'Siya.wav' uploaded successfully.")
else:
  print("Please upload 'Siya.wav' to proceed.")

Saving Siya.wav to Siya.wav
File 'Siya.wav' uploaded successfully.
